# Simulation with non-uniform characteristic velocity

## 1 – Non-linear velocity.

Repeat the simulation of exercise 2a, but now using a non-linear velocity $a$ of the form:

$$a(x) = -\frac{1}{2}\left\{\frac{1}{2} + 5 \cos^2 \left[\frac{4\pi}{9}s(x)\right]\exp[-4s^2(x)]\right\} \tag{1}$$

with $s(x) = -1 + 2\frac{x-x_0}{x_f-x_0}$

For the experiments, use a large number of intervals, e.g., 2048 or 4096. Since $a(x)$ is no longer uniform, the expression for $\Delta t$ used in Exercise 2a gives a different value for each point in the grid. We shall take as $\Delta t$ the minimum of all those values. For this exercise, use `nm.deriv_cent` and `cfl_cut=0.5`. Describe the most important deviations in the simulation behavior compared to when $a$ was uniform.

In [ ]:
import numpy as np
from scipy.integrate import simpson, solve_ivp
import matplotlib.pyplot as plt
from IPython.display import HTML
from importlib import reload

import import_ipynb
from nm_lib import nm_lib as nm, utils as utils
from ex_2b import get_uu_t0

plt.style.use("fast")


def get_non_a(xx: np.ndarray, x0: float = -2.6, xf: float = 2.6) -> np.ndarray:
    s = get_s(xx, x0, xf)
    costerm = np.cos(4 * np.pi / 9 * s) ** 2
    expterm = np.exp(-4 * s**2)
    return -1 / 2 * (1 / 2 + 5 * costerm * expterm)


def get_s(xx, x0, xf):
    x0 = xx[0] if x0 is None else x0
    xf = xx[-1] if xf is None else xf
    s = -1 + 2 * (xx - x0) / (xf - x0)
    return s


In [ ]:
if __name__ == "__main__":
    nint = 2048
    x0 = -2.6
    xf = 2.6
    xx, dx = utils.get_xx(nint, x0, xf)
    uut0 = get_uu_t0(xx)
    a_non = get_non_a(xx)
    a_uni = uut0
    plt.plot(xx, a_non, label="non")
    plt.plot(xx, a_uni, label="uni")
    plt.legend()


In [ ]:
if __name__ == "__main__":
    tt_non, uunt_non = nm.evolv_Lax_adv_burgers(
        xx, uut0, a_non, end_time=10, cfl_cut=0.5, ddx=lambda x, u: nm.deriv_cent(x, u), bnd_limits=[1, 1]
    )
    tt_uni, uunt_uni = nm.evolv_Lax_adv_burgers(
        xx, uut0, a=-1, end_time=10, cfl_cut=0.5, ddx=lambda x, u: nm.deriv_cent(x, u), bnd_limits=[1, 1]
    )


In [ ]:
if __name__ == "__main__":
    sol_dict = {"Non": (tt_non, uunt_non), "Uni": (tt_uni, uunt_uni)}
    display(HTML(utils.animate_us(sol_dict, tt_non[::100], xx).to_jshtml()))
    plt.close()
    fig, ax = plt.subplots()
    for name, (tt, uunt) in sol_dict.items():
        area = simpson(uunt[:, 1:], xx[1:])
        ax.plot(tt, area, label=name)
    ax.set_ylabel(r"$\int u(x,t) dx$")
    ax.set_xlabel("t")
    ax.legend()


## 2 – Interpretation of the result of the experiment.

To understand the result, we must recall the fundamental property of the linear advection equation, namely: the value of the solution $u(x, t)$ stays constant if we move along the $x$ axis with velocity $a$, i.e., following the curves $x_p(t)$ defined through:

$$\frac{d x_p}{dt} = a[x_p(t),t]  \tag{2}$$

1. If you know how to integrate ordinary differential equations (or, more simply, in this case, how to calculate numerical quadratures), find, using Python, the function $x_p(t)$ that solves equation (2) with initial condition $x_p(t = 0) = x_0$. Then do the same for $x_p(t = 0) = x_f$ . Then, once you have the program working for you, it is easy to calculate the $x_p(t)$ function starting from points in the interior of the interval. With those, draw the solutions for a number of initial representative values.

In [ ]:
if __name__ == "__main__":
    rhs = lambda t, x: get_non_a(utils.get_periodic_value(x0, xf, x))[:, ...]
    tf = 20
    tt = np.linspace(0, tf, 100)
    x_init = np.linspace(x0, xf, 7)
    markers = [".", "1", "2", "3", "4", "+", "x", "*"]
    sol = solve_ivp(rhs, [0, tf], x_init, dense_output=True, vectorized=True)

    fig, (pax, ax) = plt.subplots(nrows=2, sharex=True)
    for i, xp in enumerate(sol.sol(tt)):
        pax.plot(
            tt,
            utils.get_periodic_value(x0, xf, xp),
            marker=markers[i % len(markers)],
            linestyle="None",
            label=f"$x_p(0)={x_init[i]:.2f}$",
        )
        ax.plot(tt, xp, marker=markers[i % len(markers)], linestyle="None")
    pax.legend(bbox_to_anchor=(0.0, 1.02, 1.0, 0.102), loc="lower left", mode="expand", ncol=3)
    pax.axhline(y=x0, ls=":", color="k")
    pax.axhline(y=xf, ls=":", color="k")
    pax.set_ylabel("$x_p(t)$, periodic")
    ax.set_ylabel("$x_p(t)$")
    ax.set_xlabel("$t$")


2. Based on the solutions you have just found for $x_p$, interpret the time evolution you see in the PDE solution of paragraph  1.